In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time

In [28]:
url = 'https://store.steampowered.com/search/results/?query&start=0&count=50&dynamic_data=&sort_by=_ASC&tags=4182&snr=1_7_7_2300_7&specials=1&filter=topsellers&infinite=1'

def totalresults(url):
    r = requests.get(url)
    data = dict(r.json())
    totalresults = data['total_count']
    return int(totalresults)

def get_data(url):
    r = requests.get(url)
    data = dict(r.json())
    return data['results_html']

print (get_data(url))


<!-- List Items -->

				<a href="https://store.steampowered.com/bundle/15295/Battlefield_Bundle/?snr=1_7_7_2300_150_1"
			 data-ds-bundleid="15295" data-ds-bundle-data="{&quot;m_nDiscountPct&quot;:&quot;25&quot;,&quot;m_bMustPurchaseAsSet&quot;:0,&quot;m_rgItems&quot;:[{&quot;m_nPackageID&quot;:432910,&quot;m_rgIncludedAppIDs&quot;:[1238860],&quot;m_bPackageDiscounted&quot;:true,&quot;m_nBasePriceInCents&quot;:15900,&quot;m_nFinalPriceInCents&quot;:1908,&quot;m_nFinalPriceWithBundleDiscount&quot;:1431},{&quot;m_nPackageID&quot;:432909,&quot;m_rgIncludedAppIDs&quot;:[1238840],&quot;m_bPackageDiscounted&quot;:true,&quot;m_nBasePriceInCents&quot;:15900,&quot;m_nFinalPriceInCents&quot;:1908,&quot;m_nFinalPriceWithBundleDiscount&quot;:1431},{&quot;m_nPackageID&quot;:505826,&quot;m_rgIncludedAppIDs&quot;:[1238810],&quot;m_bPackageDiscounted&quot;:true,&quot;m_nBasePriceInCents&quot;:24900,&quot;m_nFinalPriceInCents&quot;:2490,&quot;m_nFinalPriceWithBundleDiscount&quot;:1867}],&quot;m_bIsCo

In [74]:
def parse(data):
    gameslist = []
    soup = BeautifulSoup(data, 'html.parser')
    games = soup.find_all('a')
    for game in games:
        title = game.find('span', {'class': 'title'}).text
        try:
            date_released = game.find('div', {'class' : 'search_released'}).text
        except:
            date_released = ' '
        original_price = game.find('div', {'class': 'search_price'}).text.strip().split('RM')[1]
        try:
            discount_price = game.find('div', {'class': 'search_price'}).text.strip().split('RM')[2]
        except:
            discount_price = original_price
        try:
            review = game.find('span', {'data-tooltip-html': True})["data-tooltip-html"]
        except:
            review = ' '
        print(title, date_released, original_price, discount_price, review)

        mygame ={
            'title': title,
            'date_released': date_released,
            'original_price': original_price,
            'discount_price': discount_price,
            'review': review
        }
        gameslist.append(mygame)
    return gameslist

In [78]:
def output(results):
    gamesdf = pd.concat([pd.DataFrame(g) for g in results])
    gamesdf.to_csv('SteamTopSellerSpecialOfferPrices_Singleplayer.csv', index=False)
    print('Fin. Saved to CSV')
    print(gamesdf.head())
    return

In [76]:
results = []
for x in range(0, totalresults(url), 50):
    data = get_data(f'https://store.steampowered.com/search/results/?query&start={x}&count=50&dynamic_data=&sort_by=_ASC&tags=4182&snr=1_7_7_2300_7&specials=1&filter=topsellers&infinite=1')
    results.append(parse(data))
    print('Results Scraped: ', x)
    time.sleep(1.5)

Battlefield Bundle  567.00 47.29 Mostly Positive<br>78% of the 129,057 user reviews for games in this bundle are positive.
Fallout 4: Game of the Year Edition 10 Nov, 2015 167.48 41.87 Mostly Positive<br>78% of the 170,391 user reviews for games in this bundle are positive.
Battlefield V 22 Oct, 2020 249.00 24.90 Mixed<br>69% of the 59,724 user reviews for this game are positive.
Disco Elysium - The Final Cut 15 Oct, 2019 65.00 35.75 Very Positive<br>94% of the 38,483 user reviews for this game are positive.
Gladiator Guild Manager 13 Oct, 2021 25.00 22.50 Very Positive<br>89% of the 77 user reviews for this game are positive.
Football Manager 2022 9 Nov, 2021 159.00 143.10  
House Flipper 17 May, 2018 39.00 19.50 Very Positive<br>92% of the 43,399 user reviews for this game are positive.
Growing Up 13 Oct, 2021 32.00 28.80 Positive<br>83% of the 12 user reviews for this game are positive.
Assassin's Creed® Odyssey 5 Oct, 2018 189.00 47.25 Very Positive<br>89% of the 95,538 user review

Ys IX: Monstrum Nox Digital Ultimate Edition  209.98 137.75 Overwhelmingly Positive<br>96% of the 691 user reviews for games in this bundle are positive.
Children of Morta 3 Sep, 2019 44.00 17.60 Very Positive<br>91% of the 8,262 user reviews for this game are positive.
Assassin's Creed® Revelations 30 Nov, 2011 45.00 14.85 Very Positive<br>85% of the 9,973 user reviews for this game are positive.
9-nine-  156.00 98.28 Overwhelmingly Positive<br>96% of the 1,849 user reviews for games in this bundle are positive.
R-Type Final 2 Digital Deluxe Edition  149.50 107.64 Mostly Positive<br>76% of the 380 user reviews for games in this bundle are positive.
Baldur's Gate: Enhanced Edition 16 Jan, 2013 38.00 7.60 Very Positive<br>91% of the 9,263 user reviews for this game are positive.
Gladiators Tavern Bundle  57.00 49.05 Very Positive<br>92% of the 1,559 user reviews for games in this bundle are positive.
Cooking Simulator VR 29 Jul, 2021 44.00 35.20 Very Positive<br>88% of the 385 user revi

Metal Unit 27 Jan, 2021 32.00 17.60 Very Positive<br>84% of the 1,021 user reviews for this game are positive.
Assault Spy 2 Oct, 2018 39.00 15.60 Very Positive<br>91% of the 419 user reviews for this game are positive.
Phantom Brave PC 25 Jul, 2016 38.00 5.70 Very Positive<br>89% of the 354 user reviews for this game are positive.
Victoria Collection  162.00 43.16 Very Positive<br>91% of the 11,238 user reviews for games in this bundle are positive.
Mr. Prepper in the Endzone  98.00 75.68 Mostly Positive<br>79% of the 6,699 user reviews for games in this bundle are positive.
XCOM 2: War of the Chosen 3 Dec, 2018 119.00 29.75 Very Positive<br>83% of the 3,035 user reviews for this game are positive.
Shelter Manager 8 Oct, 2021 32.00 28.80 Mostly Positive<br>78% of the 92 user reviews for this game are positive.
XCOM®: Chimera Squad 23 Apr, 2020 59.00 29.50 Mostly Positive<br>73% of the 16,685 user reviews for this game are positive.
Gold Rush: The Game - Parker's Edition  75.00 33.74 M

HALF DEAD 3 20 Aug, 2021 12.50 10.00 Mostly Positive<br>72% of the 116 user reviews for this game are positive.
Caveman World: Mountains of Unga Boonga 26 Apr, 2016 2.70 1.51 Mixed<br>66% of the 350 user reviews for this game are positive.
REKKR: Sunken Land 11 Oct, 2021 17.50 12.42 Positive<br>100% of the 11 user reviews for this game are positive.
Assault Spy Elite Spy Edition (Game + Soundtrack)  49.50 18.81 Very Positive<br>92% of the 427 user reviews for games in this bundle are positive.
Thief Simulator VR 5 Nov, 2020 39.00 23.40 Mixed<br>68% of the 678 user reviews for this game are positive.
This Is the Police Bundle  62.00 13.95 Very Positive<br>81% of the 10,973 user reviews for games in this bundle are positive.
West Sweety 13 Dec, 2019 5.50 4.12 Very Positive<br>86% of the 2,508 user reviews for this game are positive.
Disgaea 2 PC Digital Dood Edition (Game + Art Book)  50.00 9.00 Very Positive<br>91% of the 1,053 user reviews for games in this bundle are positive.
GameGur

ARIA CHRONICLE Digital Deluxe Edition  86.50 54.15 Mostly Positive<br>79% of the 1,027 user reviews for games in this bundle are positive.
【礼包】嗜血印本体+全部服装DLC  262.50 101.50 Very Positive<br>86% of the 17,380 user reviews for games in this bundle are positive.
Novus Inceptio 5 Oct, 2015 30.00 9.00 Mixed<br>57% of the 947 user reviews for this game are positive.
Farmer Life Simulator 21 Aug, 2021 39.00 19.50 Mixed<br>48% of the 43 user reviews for this game are positive.
Anomaly 2 15 May, 2013 31.00 6.20 Very Positive<br>80% of the 301 user reviews for this game are positive.
Lust for the Beast  88.00 48.07 Very Positive<br>85% of the 3,942 user reviews for games in this bundle are positive.
Fantasy Blacksmith 15 Nov, 2019 23.50 15.74 Mixed<br>69% of the 735 user reviews for this game are positive.
GameGuru SciFi Bundle 25 Jan, 2016 59.00 17.70 Mostly Positive<br>73% of the 1,116 user reviews for games in this bundle are positive.
Paws 24 Mar, 2016 31.00 6.20 Very Positive<br>90% of the 4

...

Dark Zone Defense 29 Oct, 2019 5.50 1.54 Mostly Positive<br>78% of the 14 user reviews for this game are positive.
Indie-Games.Studio Bundle  356.50 89.87 Mixed<br>51% of the 549 user reviews for games in this bundle are positive.
Raccoon Arrival 12 Oct, 2021 19.50 12.67  
Avadon 3: The Warborn 14 Sep, 2016 38.00 11.40 Very Positive<br>91% of the 85 user reviews for this game are positive.
Flynguin Station 24 Sep, 2019 12.50 3.75 Positive<br>85% of the 20 user reviews for this game are positive.
Starlight Alliance 13 Oct, 2021 23.50 19.97  
Tortured Hearts - Or How I Saved The Universe. Again. 1 Mar, 2021 51.00 30.60 Positive<br>80% of the 21 user reviews for this game are positive.
Camp W 25 Jul, 2018 12.50 2.50 Positive<br>93% of the 16 user reviews for this game are positive.
8-Bit Complete Collection  132.99 98.54 Very Positive<br>81% of the 2,474 user reviews for games in this bundle are positive.
Science Girls 20 Mar, 2014 12.00 3.00 Mostly Positive<br>71% of the 118 user reviews

Ironcast Complete Collection  51.00 28.00 Mostly Positive<br>75% of the 708 user reviews for games in this bundle are positive.
Desktop NEKO Girl 21 Jan, 2021 15.50 10.85 Positive<br>80% of the 31 user reviews for this game are positive.
VR Shooters bundle  49.50 18.90 Mostly Positive<br>78% of the 46 user reviews for games in this bundle are positive.
Bomb Bowling 22 Aug, 2019 12.50 1.50  
Wordle 2 31 Aug, 2020 5.50 1.54 Very Positive<br>92% of the 80 user reviews for this game are positive.
Wooden House 5 Sep, 2016 3.00 1.50 Mostly Negative<br>32% of the 100 user reviews for this game are positive.
Crossy Bridge 28 Apr, 2021 3.00 1.65  
Conundrum 8 Oct, 2021 10.50 9.45  
1001 Jigsaw Legends of Mystery 2 6 Jul, 2021 12.50 1.50  
Vortex Rolling 14 Jun, 2021 3.00 1.65  
Cross Board SOCCER 14 Jul, 2021 3.00 1.65  
Ancient Frontier 21 Sep, 2017 47.00 9.40 Mixed<br>64% of the 95 user reviews for this game are positive.
Asian Riddles 4 2 Feb, 2021 12.50 1.50  
Shan Gui:	Sweet Osmanthus Seri

Sheptun 13 Feb, 2020 3.00 1.65 Positive<br>85% of the 14 user reviews for this game are positive.
Centralia: Homecoming 26 Oct, 2019 30.00 10.50 Mixed<br>66% of the 42 user reviews for this game are positive.
Barro Collection  12.00 8.92 Very Positive<br>83% of the 2,850 user reviews for games in this bundle are positive.
Vagrant Hearts 27 Feb, 2015 25.00 2.50 Mixed<br>48% of the 50 user reviews for this game are positive.
Everybody Wham Wham 29 Jan, 2021 19.50 14.62 Positive<br>96% of the 26 user reviews for this game are positive.
Chocolate makes you happy 4 14 Mar, 2018 3.00 1.50 Mostly Positive<br>76% of the 30 user reviews for this game are positive.
SUDOKU TIME 22 Sep, 2020 5.50 1.54  
Space Hero Line 24 Aug, 2017 3.00 1.50 Mixed<br>56% of the 39 user reviews for this game are positive.
Mr. Parkour 2 13 Nov, 2020 10.50 1.57  
Chocolate makes you happy 5 24 Apr, 2018 3.00 1.50 Positive<br>87% of the 24 user reviews for this game are positive.
Chocolate makes you happy 3 6 Feb, 201

Mysterious Blocks 2 17 Jun, 2021 10.50 1.57 Positive<br>83% of the 12 user reviews for this game are positive.
Dice Tourney 16 Aug, 2021 10.50 2.62 Mixed<br>62% of the 16 user reviews for this game are positive.
Martian Potato 16 Jun, 2020 12.50 3.75  
VR Another World 11 Jun, 2021 12.50 6.25  
task: 312 25 Aug, 2021 12.50 6.25  
1001 Jigsaw Home Sweet Home Wedding Ceremony 26 Jan, 2021 12.50 1.50  
Holiday Mosaics Christmas Puzzles 9 Jun, 2020 12.50 1.50  
Butterfly 21 Feb, 2019 12.50 1.50 Positive<br>95% of the 20 user reviews for this game are positive.
1406 21 Mar, 2019 15.50 3.87 Mixed<br>45% of the 61 user reviews for this game are positive.
JQ: Beautiful Japan 23 Aug, 2018 2.96 1.50 Mostly Positive<br>71% of the 67 user reviews for this game are positive.
Trivia Vault: Video Game Trivia Deluxe 24 Oct, 2017 59.99 5.99 Mixed<br>58% of the 39 user reviews for this game are positive.
USAGIRI 11 Dec, 2019 23.50 2.35 Positive<br>87% of the 32 user reviews for this game are positive.
S

Underground Keeper 15 Aug, 2016 23.50 5.87 Mixed<br>68% of the 35 user reviews for this game are positive.
ZenFarm 18 Feb, 2021 5.50 1.54  
The Story Matters III  29.25 4.03 Very Positive<br>88% of the 520 user reviews for games in this bundle are positive.
Chamber of Darkness 10 Oct, 2018 7.00 1.54 Mixed<br>41% of the 29 user reviews for this game are positive.
Happy Empire - A Bouquet for the Princess 14 Jan, 2018 12.50 3.75 Mixed<br>62% of the 64 user reviews for this game are positive.
Puzzle Chambers 1 Dec, 2017 6.00 1.50 Positive<br>86% of the 43 user reviews for this game are positive.
Clessidra 15 Jun, 2021 12.50 6.25  
StOoPiD Puzzle Game 23 Jul, 2021 5.50 3.85  
TenebriS 23 Dec, 2020 36.00 3.60  
Panda in the clouds 23 Aug, 2021 5.50 3.57 Positive<br>100% of the 11 user reviews for this game are positive.
STALINGRAD ABATIS 21 Sep, 2018 3.00 1.50 Mixed<br>43% of the 16 user reviews for this game are positive.
Data mining 6 Aug, 2018 5.50 1.54 Mixed<br>50% of the 10 user review

In [79]:
output(results)

Fin. Saved to CSV
                                 title date_released original_price  \
0                   Battlefield Bundle                       567.00   
1  Fallout 4: Game of the Year Edition  10 Nov, 2015         167.48   
2                        Battlefield V  22 Oct, 2020         249.00   
3        Disco Elysium - The Final Cut  15 Oct, 2019          65.00   
4              Gladiator Guild Manager  13 Oct, 2021          25.00   

  discount_price                                             review  
0          47.29  Mostly Positive<br>78% of the 129,057 user rev...  
1          41.87  Mostly Positive<br>78% of the 170,391 user rev...  
2          24.90  Mixed<br>69% of the 59,724 user reviews for th...  
3          35.75  Very Positive<br>94% of the 38,483 user review...  
4          22.50  Very Positive<br>89% of the 77 user reviews fo...  
